In [3]:
# Sorted Class List
import os

have_ids = []

for dir in os.listdir('train/images'):
    have_ids.append(str(dir))
    
data = open('words.txt','r').read()

classes = {}
for line in data.split("\n"):    
    imagenet_id, imagenet_desc = line.split("\t")
    classes[imagenet_id] = imagenet_desc
    
have_classes = {}

for imagenet_id in have_ids:
    have_classes[imagenet_id] = classes[imagenet_id]
    
have_classes_list = []
for imagenet_id in have_classes:
    have_classes_list.append(have_classes[imagenet_id].lower())
    
have_classes_list = sorted(have_classes_list)

output = ""
for desc in have_classes_list:
    output += desc + "\n"

open('sorted_classes.txt', 'w').write(output)

21675

In [1]:
# Convert from ImageNet XML to DIGITS format

import xmltodict
import os
import collections

def imagenet_to_digits_detect(source, dest, classes, extension_in_filename=False):
    have_ids = []

    for dir in os.listdir(source):
        have_ids.append(str(dir))

    class_xml = {}

    for imagenet_id in have_ids:
        xml_files = ["%s/%s/%s" % (source, imagenet_id, str(x)) 
                     for x in os.listdir('%s/%s' % (source, imagenet_id))]
        for xml_file in xml_files:
            doc = xmltodict.parse(open(xml_file, 'r').read())['annotation']
            
            file_name = doc['filename']
            if extension_in_filename:
                parts = doc['filename'].split(".")
                file_name = ".".join(parts[:len(parts)-1])
            
            objects = doc['object']
            
            if(type(objects) == collections.OrderedDict):
                objects = [objects]
            
            digits_str = ""
            for object in objects:

                name = object['name']

                xmin = int(object['bndbox']['xmin'])
                xmax = int(object['bndbox']['xmax'])
                ymin = int(object['bndbox']['ymin'])
                ymax = int(object['bndbox']['ymax'])
                
                truncated = float(object['truncated'])

                digits_str += "%s %.1f 0 0.0 %.2f %.2f %.2f %.2f 0.0 0.0 0.0 0.0 0.0 0.0 0.0\n" % (name, truncated, xmin, ymin, xmax, ymax)
            
            digits_str = digits_str.strip()
            
            try:
                os.mkdir('%s/%s' % (dest, imagenet_id))
            except FileExistsError:
                pass

            open('%s/%s/%s.txt' % (dest, imagenet_id, file_name), 'w').write(digits_str)
            
data = open('words.txt','r').read()

classes = {}
for line in data.split("\n"):    
    imagenet_id, imagenet_desc = line.split("\t")
    classes[imagenet_id] = imagenet_desc
    

imagenet_to_digits_detect("imagenet_xml/val", "val", classes, extension_in_filename=True)
imagenet_to_digits_detect("imagenet_xml/train", "train/labels", classes, extension_in_filename=True)


In [1]:
import os
from shutil import copy
import struct
import cv2

# Create DetectNet Dataset
dataset_name = "squirrel"
classes = ['n02356798']

max_width = 1248
max_height = 384

max_area = 200*200

os.mkdir("datasets/%s" % dataset_name)
os.mkdir("datasets/%s/train" % dataset_name)
os.mkdir("datasets/%s/train/labels" % dataset_name)
os.mkdir("datasets/%s/train/images" % dataset_name)

os.mkdir("datasets/%s/val" % dataset_name)
os.mkdir("datasets/%s/val/labels" % dataset_name)
os.mkdir("datasets/%s/val/images" % dataset_name)

def process_image(source_label_path, source_image_path, dest_label_path, dest_image_path):

        image_obj = cv2.imread(source_image_path)
        height, width, channels = image_obj.shape
        
        label_data = open(source_label_path, "r").read().split("\n")
        
        if(len(label_data) == 0):
            return
        
        reduce_max_area = False
        reduce_ratio = 1
        
        for line in label_data:
            fields = line.split(" ")
            
            x1 = float(fields[4])
            y1 = float(fields[5])
            x2 = float(fields[6])
            y2 = float(fields[7])

            cal_width = x2-x1
            cal_height = y2-y1
            cal_area = cal_width * cal_height

            if cal_area > max_area:
                reduce_max_area = True
                reduce_ratio = max(cal_area/max_area, reduce_ratio)

        # We need to resize the image
        if width > max_width or height > max_height or reduce_max_area:
            
            width_ratio = width/max_width
            height_ratio = height/max_height
            
            max_ratio = max(width_ratio, height_ratio, reduce_ratio)
            
            new_width = min(int(width/max_ratio), max_width)
            new_height = min(int(height/max_ratio), max_height)

            resized_image_obj = cv2.resize(image_obj, (new_width, new_height), interpolation = cv2.INTER_CUBIC)

            cv2.imwrite(dest_image_path, resized_image_obj)
            
            output_data = ""
            for line in label_data:
                fields = line.split(" ")
                
                x1 = float(fields[4])/max_ratio
                y1 = float(fields[5])/max_ratio
                x2 = float(fields[6])/max_ratio
                y2 = float(fields[7])/max_ratio
                
                cal_width = x2-x1
                cal_height = y2-y1
                cal_area = cal_width * cal_height

                fields[4] = "%.2f" % x1
                fields[5] = "%.2f" % y1
                fields[6] = "%.2f" % x2
                fields[7] = "%.2f" % y2

                output_data += " ".join(fields) + "\n"
                
            output_data = output_data.strip()
            print(output_data)

            label_output_file = open(dest_label_path, "w")
            label_output_file.write(output_data)
        else:
            copy(source_image_path, dest_image_path)
            copy(source_label_path, dest_label_path)
    
    
for wnid in classes:
    
    print("Processing %s" % wnid)
    labels = os.listdir("train/labels/%s" % wnid)
    
    for label in labels:
                
        # ImageNet format
        name, _ = label.split(".")
        image = "%s.JPEG" % name

        process_image("train/labels/%s/%s" % (wnid, label), 
                      "train/images/%s/%s" % (wnid, image),
                      "datasets/%s/train/labels/%s" % (dataset_name, label),
                      "datasets/%s/train/images/%s" % (dataset_name, image))

for label in os.listdir('val/labels'):
        name, ext = label.split(".")
        
        image = "%s.JPEG" % name
        
        label_data = open("val/labels/%s" % (label), 'r').read().split("\n")
        
        have = False
        for wnid in classes:
            for line in label_data:
                if line.find(wnid) != -1:
                    have = True
                    break
            if have:
                break
        
        if not have:
            continue
                
        process_image("val/labels/%s" % (label),
                      "val/images/%s" % (image),
                      "datasets/%s/val/labels/%s" % (dataset_name, label),
                      "datasets/%s/val/images/%s" % (dataset_name, image))


Processing n02356798
n02356798 0.0 0 0.0 349.80 24.43 482.69 237.44 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 31.27 42.99 163.18 236.46 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 4.37 18.68 139.51 136.33 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 4.46 3.71 92.37 116.39 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 196.80 177.60 264.80 251.20 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 26.22 17.25 150.06 128.67 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 13.37 11.52 136.44 161.34 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 172.10 41.93 356.43 231.51 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 82.52 0.00 210.73 229.89 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 2.89 6.51 119.17 89.50 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 1.42 6.84 109.23 94.37 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 13.08 18.68 129.35 179.32 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 25.31 18.02 149.92 141.10 0.0 0.0 0.0 0.0 0.0 0.0 0.0
n02356798 0.0 0 0.0 34.6